In [1]:
# ライブラリの読み込み
import codecs
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

In [2]:
# CSVファイルを読み込んでXに格納
# 事前に処理したこと
## 建築年を築年数に変換
## 最寄駅：距離（分）を数値に変換
## 種類=中古マンション以外を削除
## その他不要な列を削除
with codecs.open('fudosan/34_Hiroshima Prefecture_20101_20204_mansion.csv', 'r', 'shift_jisx0213', 'ignore') as filename:
    X = pd.read_csv(filename)

In [3]:
# 読み込んだデータの構造
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4085 entries, 0 to 4084
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   種類         4085 non-null   object 
 1   市区町村名      4085 non-null   object 
 2   地区名        4085 non-null   object 
 3   最寄駅：名称     4084 non-null   object 
 4   最寄駅：距離（分）  4082 non-null   float64
 5   取引価格（総額）   4085 non-null   int64  
 6   間取り        3970 non-null   object 
 7   面積（㎡）      4085 non-null   int64  
 8   築年数        4085 non-null   int64  
 9   建物の構造      3995 non-null   object 
 10  都市計画       4076 non-null   object 
 11  建ぺい率（％）    4057 non-null   float64
 12  容積率（％）     4057 non-null   float64
 13  取引時点       4085 non-null   object 
 14  改装         3814 non-null   object 
 15  取引の事情等     115 non-null    object 
dtypes: float64(3), int64(3), object(10)
memory usage: 510.8+ KB


In [4]:
# 各列で一意なデータの数
X.nunique()

種類             1
市区町村名         14
地区名          324
最寄駅：名称        86
最寄駅：距離（分）     34
取引価格（総額）     136
間取り           15
面積（㎡）         30
築年数           52
建物の構造          5
都市計画          12
建ぺい率（％）        4
容積率（％）        10
取引時点          44
改装             2
取引の事情等         3
dtype: int64

In [5]:
# 取引価格（総額）をyに移す
X.dropna(axis=0, subset=['取引価格（総額）'], inplace=True)
y = X['取引価格（総額）']
X.drop(['取引価格（総額）'], axis=1, inplace=True)

In [7]:
# 訓練データと評価データに分割
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [9]:
# すべてのカテゴリ変数をワンホットエンコーディング
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)

In [12]:
# 取引価格（総額）の統計情報
y_train.describe()

count    3.268000e+03
mean     1.805174e+07
std      1.059953e+07
min      3.000000e+06
25%      1.100000e+07
50%      1.600000e+07
75%      2.400000e+07
max      9.500000e+07
Name: 取引価格（総額）, dtype: float64

In [14]:
# XGBoostモデルの定義
my_model_1 = XGBRegressor(random_state=0)

# 訓練データの学習
my_model_1.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [15]:
# 学習済みモデルを使って、評価データから取引価格（総額）を予測
predictions_1 = my_model_1.predict(X_valid)

In [16]:
# 平均絶対誤差の取得
mae_1 = mean_absolute_error(predictions_1, y_valid)
print("Mean Absolute Error:" , mae_1)

Mean Absolute Error: 2597357.9143206854
